## Cargamos las librerías necesarias y el dataset el cual debe contener una matriz de descriptores con el identificador y el endpoint (CAS y Tox en este caso, respectivamente).  <span style="color:red">El dataset debe ya estar limpio (sin valores constantes ni altamente correlacionados)</span>


In [17]:
import random
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score

# cargar el dataset
df = pd.read_csv(r"C:\Users\quant\Desktop\matriz.csv")



## Verificamos las primeras 10 filas del dataset, no vaya ser que la cagamos. 

In [18]:
df.head(10)

,CAS,Tox,MW,nTA,RBF,C%,N%,max_conj_path,NRS,NNRS,...,phLevel2,s2_size,s3_relSize,s3_phRelSize,s2_relPathLength,s2_relPathLength_2,s3_relPathLength_2,s3_numRotBonds,s4_numRotBonds,s4_numAroBonds
0,84-75-3,1.05,334.50,4,0.259259,37.037037,0.000000,10,1,1.0,...,0,0,0.000000,0.0,0.000000,0.0,0.0,0,0,0
1,89-19-0,1.40,362.56,4,0.266667,36.666667,0.000000,10,1,1.0,...,0,0,0.000000,0.0,0.000000,0.0,0.0,0,0,0
2,105-28-2,1.99,308.62,1,0.258065,32.258065,3.225806,0,1,1.0,...,0,0,0.000000,0.0,0.000000,0.0,0.0,0,0,0
3,117-81-7,1.12,390.62,6,0.242424,36.363636,0.000000,10,1,1.0,...,1,2,0.142857,0.0,0.071429,1.0,1.0,2,11,6
4,117-84-0,0.92,390.62,4,0.272727,36.363636,0.000000,10,1,1.0,...,0,0,0.000000,0.0,0.000000,0.0,0.0,0,0,0
5,119-07-3,0.98,418.68,4,0.277778,36.111111,0.000000,10,1,1.0,...,0,0,0.000000,0.0,0.000000,0.0,0.0,0,0,0
6,123-95-5,1.03,340.66,3,0.298507,32.352941,0.000000,0,0,0.0,...,0,0,0.000000,0.0,0.000000,0.0,0.0,0,0,0
7,137-89-3,1.36,390.62,6,0.242424,36.363636,0.000000,10,1,1.0,...,1,2,0.142857,0.0,0.071429,1.0,1.0,2,11,6
8,1928-43-4,3.05,333.28,5,0.232558,37.209302,0.000000,6,1,1.0,...,1,2,0.190476,0.0,0.095238,1.0,1.0,2,5,6
9,1928-45-6,2.83,335.25,4,0.268293,36.585366,0.000000,6,1,1.0,...,0,0,0.000000,0.0,0.000000,0.0,0.0,0,0,0


## Separamos los números CAS y el endpoint y dividimos el dataset aleatoriamente en una proporción 70/30. 

In [19]:
# separar las características y la respuesta
X = df.drop(['CAS', 'Tox'], axis=1).values
y = df['Tox'].values

# dividir los datos en conjuntos de entrenamiento y prueba (70/30)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

# separar las características y la respuesta
X_train = df.drop(['CAS', 'Tox'], axis=1).values
y_train = df['Tox'].values



## Echamos a andar el algoritmo genético

In [20]:
# Inicializamos los parámetros del algoritmo genético
POPULATION_SIZE = 100
NUM_GENERATIONS = 50
MUTATION_RATE = 0.1

# Aquí definimos  una función de evaluación para el modelo de regresión lineal. 
'''
Aquí escogí evaluar el modelo por medio de su R2 y su Q2 de validación externa, pero tú  puedes
imponer otro criterio , por ejemplo RMSE, s, F, o algún otro. 
'''
def evaluate_fitness(features):
    X_selected = X[:, features]
    X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.3)
    model = LinearRegression()
    model.fit(X_train, y_train)
    r2 = r2_score(y_test, model.predict(X_test))
    q2 = cross_val_score(model, X_selected, y, cv=5, scoring='r2').mean()
    return r2, q2

# Creamos una población inicial de cromosomas
population = []
for i in range(POPULATION_SIZE):
    chromosome = random.sample(range(X.shape[1]), 6)
    population.append(chromosome)

# Aquí iteremos sobre las generaciones
for generation in range(NUM_GENERATIONS):

    # Aquí vamos a guardar los Fitness
    fitness_scores = []
    for chromosome in population:
        fitness_scores.append(evaluate_fitness(chromosome))

    # Con base a los mejores fitness , elegimos a los más chidos 
    elite_indices = sorted(range(len(fitness_scores)), key=lambda k: sum(fitness_scores[k]), reverse=True)[:int(POPULATION_SIZE*0.2)]
    elite_population = [population[i] for i in elite_indices]

    # reproducir la población elite
    new_population = elite_population.copy()
    while len(new_population) < POPULATION_SIZE:
        parent1, parent2 = random.choices(elite_population, k=2)
        crossover_point = random.randint(1, 5)
        child = sorted(parent1[:crossover_point] + [x for x in parent2 if x not in parent1[:crossover_point] and len(parent1[:crossover_point] + [x for x in parent2 if x not in parent1[:crossover_point]]) < 6])
        
        while len(child) < 6:
            new_gene = random.choice([x for x in range(X.shape[1]) if x not in child])
            if new_gene not in child:
                child.append(new_gene)
        child = sorted(child)

        if random.random() < MUTATION_RATE:
            mutation_point = random.choice(range(6))
            new_gene = random.choice([x for x in range(X.shape[1]) if x not in child])
            child[mutation_point] = new_gene
            child = sorted(child)

        new_population.append(child)

    # actualizar la población
    population = new_population

# Calculamos el fitness de cada cromosoma final
fitness_scores = []
for chromosome in population:
        fitness_scores.append(evaluate_fitness(chromosome))

# seleccionamos los 10 mejores modelos, basado en los criterios que ya dije allá arriba.
top_models = sorted(range(len(fitness_scores)), key=lambda k: sum(fitness_scores[k]), reverse=True)[:10]



## Vamos a imprimir los subsets quejor predecirían la respuesta en caso de generar un modelo OLS con ellos. 

In [21]:
# imprimir los resultados
for i in top_models:
    features = population[i]
    r2, q2 = fitness_scores[i]
    print(f"Features: {df.columns[2:][features]}")
    print(f"R2: {r2:.3f}")
    print(f"Q2: {q2:.3f}")
    print()

Features: Index(['rGes', 'SpMax2_Bh(s)', 'SpMax7_Bh(s)', 'SpMin4_Bh(s)', 'Chi0_EA(dm)',
       'SdssC'],
      dtype='object')
R2: 0.957
Q2: 0.818

Features: Index(['rGes', 'SpMax2_Bh(s)', 'SpMax7_Bh(s)', 'Chi0_EA(dm)', 'Eig09_AEA(dm)',
       'SdssC'],
      dtype='object')
R2: 0.926
Q2: 0.834

Features: Index(['rGes', 'SpMax2_Bh(s)', 'SpMax7_Bh(s)', 'Chi0_EA(dm)', 'SdssC',
       'F08[C-N]'],
      dtype='object')
R2: 0.917
Q2: 0.835

Features: Index(['rGes', 'SpMax2_Bh(s)', 'SpMax7_Bh(s)', 'Chi0_EA(dm)', 'SdssC', 'RNCG'], dtype='object')
R2: 0.903
Q2: 0.837

Features: Index(['rGes', 'SpMax2_Bh(s)', 'SpMax7_Bh(s)', 'Chi0_EA(dm)', 'SdssC',
       's2_size'],
      dtype='object')
R2: 0.940
Q2: 0.788

Features: Index(['SpDiam_B(m)', 'ATS3s', 'GATS4m', 'GGI1', 'SpAD_AEA(ed)',
       'Chi0_EA(dm)'],
      dtype='object')
R2: 0.921
Q2: 0.799

Features: Index(['rGes', 'GATS2e', 'SpMax2_Bh(s)', 'SpMax7_Bh(s)', 'Chi0_EA(dm)',
       'SdssC'],
      dtype='object')
R2: 0.903
Q2: 0.817

Featur